# Experiment: VAE
**Beschrijving:** Het model werkt nu niet meer, omdat de data is veroudert, ik wil het model opnieuw trainen met nieuwe data om te kijken als het dan wel weer werkt.

**Datum:** 27 Janurai 2025

**Auteur:** Paul

**Versie:** V0.2

**Status:** Completed

## Doel en Verwachtingen
**Doelstelling:** Kijken als het mogelijk is om het probleem op te lossen.

**Verwachting:** Ik hoop het

## Data en Bronnen
**Datasets:**
- N.v.t. dataset is lokaal opgenomen

**Externe Bronnen:**
- N.v.t.

## Software Configuratie
**Software:**

In [1]:
# Data Manipulation and Analysis
import pandas as pd # type: ignore
import numpy as np # type: ignore
import librosa

# Machine Learning and Data Preprocessing
from tensorflow.keras.layers import Input, Dense, Lambda, Conv2D, Conv2DTranspose, Flatten, Reshape # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.losses import mse # type: ignore
import tensorflow.keras.backend as K # type: ignore
# Visualization
import matplotlib.pyplot as plt # type: ignore

# System and File Management
import sounddevice as sd # type: ignore
import sys
import os
import time

# Add a path to the scripts directory
sys.path.append(os.path.abspath(os.path.join('../../', 'scripts')))

# Project-Specific Modules
from RetrievingData import load_dataframe, load_dataset # type: ignore
from AnalysisPlots import plot_label_distribution # type: ignore
from ProcessingData import process_audio # type: ignore

## Business Understanding
**Doel:** Een model dat werkt op basis van data dat als normaal wordt geschouwd in een specifieke context.

**Succescriterium:** Een model dat werkt met normale data.

## Data Understanding
**Datasets:** Zelf opgenomen data van mijn kantoor thuis.

In [2]:
desired_labels = ['kantoor_2025']
df, label_mapping = load_dataframe('data/', desired_labels)

df

,file_path,label,label_numerical
0,data/kantoor_2025\opname_20250127_110820.wav,kantoor_2025,0
1,data/kantoor_2025\opname_20250127_110822.wav,kantoor_2025,0
2,data/kantoor_2025\opname_20250127_110824.wav,kantoor_2025,0
3,data/kantoor_2025\opname_20250127_110826.wav,kantoor_2025,0
4,data/kantoor_2025\opname_20250127_110828.wav,kantoor_2025,0
...,...,...,...
145,data/kantoor_2025\opname_20250127_111319.wav,kantoor_2025,0
146,data/kantoor_2025\opname_20250127_111321.wav,kantoor_2025,0
147,data/kantoor_2025\opname_20250127_111323.wav,kantoor_2025,0
148,data/kantoor_2025\opname_20250127_111325.wav,kantoor_2025,0


## Data Preparation

De Short-Time Fourier Transform (STFT) is een techniek om een audio- of tijdsignaal in het frequentiedomein te analyseren door het signaal op te splitsen in korte segmenten (vensters) over de tijd. Voor elk venster wordt een Fourier-transformatie uitgevoerd, wat de frequentie-inhoud van dat specifieke tijdssegment onthult. Dit biedt een manier om te analyseren welke frequenties aanwezig zijn op specifieke momenten in het signaal.

In ons project gebruiken we de STFT om de frequentiecomponenten van het audiosignaal over de tijd te extraheren. We zetten de amplitude van de STFT-resultaten om in een decibel-schaal, waardoor we een spectrogram verkrijgen. Dit spectrogram geeft de amplitude van de frequenties weer over de tijd, wat nuttig is voor verdere verwerking in ons model (zoals bij de reconstructie van geluiden in een autoencoder).

Waarom kiezen we voor STFT en een spectrogram in plaats van MFCC:

1. **STFT en spectrogram bieden meer flexibiliteit en detail:**
    - Een spectrogram is gebaseerd op de STFT en visualiseert de amplitude-informatie van het frequentiespectrum. Door de STFT te gebruiken en deze om te zetten naar een spectrogram, behouden we de volledige frequentie-informatie, zonder de fase, maar dit is vaak niet nodig voor onze toepassing. Dit biedt ons meer flexibiliteit en eenvoud in het verwerken van complexe audiogegevens.

2. **Meer precieze frequentieanalyse:**
    - In tegenstelling tot MFCC (Mel-Frequency Cepstral Coefficients), dat de frequentie-informatie samenvat op een manier die het menselijke gehoor nabootst (waarbij lagere frequenties gedetailleerder worden weergegeven dan hogere frequenties), biedt het spectrogram een fijnmaziger en directer overzicht van de volledige frequentieband. Dit is vooral nuttig voor anomaliedetectie, waarbij onverwachte frequenties op elk punt in het spectrum kunnen voorkomen.

3. **Geen beperking tot menselijke perceptie:**
    - MFCC is zeer geschikt voor spraakherkenning en taken gebaseerd op menselijke perceptie. Echter, voor onze toepassing in akoestische anomaliedetectie, willen we geen beperking leggen op de frequenties die we analyseren. Een spectrogram op basis van de STFT biedt een breed en gedetailleerd frequentiebereik, zonder zich te richten op alleen die frequenties die relevant zijn voor menselijke waarneming.

4. **Aanpasbaarheid van parameters:**
    - Met de STFT kunnen we de parameters zoals FFT-grootte en hop length dynamisch aanpassen, afhankelijk van de behoeften van ons project. Dit stelt ons in staat de tijd-resolutie en frequentie-resolutie te finetunen, wat belangrijk is voor de variatie in geluiden die we analyseren.

In [3]:
class AudioProcessor:
    def __init__(self, sr=44100, n_fft=1024, hop_length=512, target_length=128):
        """
        Initialize the AudioProcessor class with the specified parameters for STFT and spectrogram processing.

        Args:
            sr (int): Sample rate for audio processing.
            n_fft (int): Number of FFT components.
            hop_length (int): Number of audio samples between STFT columns.
            target_length (int): Target time dimension length for the spectrogram.
        """
        self.sr = sr
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.target_length = target_length

    def audio_to_specto(self, audio):
        """
        Compute the Short-Time Fourier Transform (STFT) and return the normalized spectrogram.

        Args:
            audio (np.ndarray): Audio signal to process.

        Returns:
            np.ndarray: Normalized spectrogram with a channel dimension added.
        """
        stft_result = librosa.stft(audio, n_fft=self.n_fft, hop_length=self.hop_length)
        spectrogram = librosa.amplitude_to_db(abs(stft_result))
        spectrogram_normalized = (spectrogram - np.min(spectrogram)) / (np.max(spectrogram) - np.min(spectrogram))

        current_length = spectrogram_normalized.shape[1]
        if current_length < self.target_length:
            pad_width = self.target_length - current_length
            spectrogram_normalized = np.pad(spectrogram_normalized, ((0, 0), (0, pad_width)), mode='constant')
        else:
            spectrogram_normalized = spectrogram_normalized[:, :self.target_length]

        spectrogram_normalized = spectrogram_normalized[..., np.newaxis]  # Add channel dimension
        return spectrogram_normalized

    def process_df(self, file_path):
        """
        Load an audio file, compute the STFT, and return the processed spectrogram.

        Args:
            file_path (str): Path to the audio file.

        Returns:
            np.ndarray: Processed spectrogram.
        """
        audio, _ = librosa.load(file_path, sr=self.sr)
        return self.audio_to_specto(audio)

    def process_audio_live(self, segment_samples):
        """
        Process live audio samples and return the processed spectrogram in the shape (1, 513, 128, 1).

        Args:
            segment_samples (list): List of live audio samples.

        Returns:
            np.ndarray: Processed spectrogram with shape (1, 513, 128, 1).
        """
        samples_np = np.array(segment_samples)
        spectrogram = self.audio_to_specto(samples_np)
        spectrogram = np.expand_dims(spectrogram, axis=0)  # Add batch dimension
        return spectrogram 

In [4]:
# Process audio files into spectrograms and resize them to a fixed length
audio_processor = AudioProcessor()

spectrograms = [
    librosa.util.fix_length(audio_processor.process_df(row['file_path']), size=128, axis=1)
    for _, row in df.iterrows()
]
X_train = np.array(spectrograms)

## Modeling

Een Variational Autoencoder (VAE) is een type autoencoder dat wordt gebruikt voor unsupervised learning, waarbij we een probabilistisch model leren dat complexe data zoals afbeeldingen, audio, of tekst kan reconstrueren. Het belangrijkste kenmerk van een VAE is dat het de latente ruimte waarin data worden geprojecteerd, dwingt om een continuüm te vormen, wat nuttig is voor taken zoals generatie van nieuwe data en anomaly detection.

--- 

We beginnen met de sampling-functie, die een steekproef neemt uit de verdeling van de latente ruimte op basis van de `z_mean` en `z_log_var`. Dit zorgt ervoor dat het model probabilistisch is, wat betekent dat het in plaats van één vaste punt meerdere mogelijke punten in de latente ruimte kan genereren.

In [5]:
# Sampling function for the latent space
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    
    # epsilon is random noise drawn from a standard normal distribution
    epsilon = K.random_normal(shape=(batch, dim))
    
    # Take a sample based on z_mean and z_log_var
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

- `z_mean` en `z_log_var` zijn de parameters die de verdeling van de latente ruimte beschrijven.
- `epsilon` introduceert willekeurigheid, wat het model in staat stelt om verschillende output te genereren vanuit dezelfde invoer.

--- 

De **VAE**-class brengt de encoder en decoder samen en berekent de totale verliesfunctie (loss), bestaande uit de reconstructiefout en de KL-divergence. Dit zorgt ervoor dat het model leert hoe het de invoer kan reconstrueren en tegelijkertijd een georganiseerde latente ruimte leert.

In [6]:
# VAE custom class
class VAE(Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        # Pass input through the encoder and obtain z_mean, z_log_var, and z (the sample)
        z_mean, z_log_var, z = self.encoder(inputs)
        
        # Pass the sample through the decoder to get the reconstruction
        reconstructed = self.decoder(z)

        # Clip the log-variance to avoid numerical instability
        z_log_var = K.clip(z_log_var, -10, 10)

        # Compute the reconstruction loss using Mean Squared Error (MSE)
        reconstruction_loss = mse(K.flatten(inputs), K.flatten(reconstructed))

        # Compute KL divergence with clipped z_log_var
        kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)

        # Combine reconstruction loss and KL divergence to form the total loss
        total_loss = K.mean(reconstruction_loss + kl_loss)

        # Add the total loss to the model
        self.add_loss(total_loss)

        return reconstructed

- Encoder en decoder werken samen om de data te coderen naar een latente ruimte (`z_mean` en `z_log_var`) en deze daarna te reconstrueren.
- Reconstructiefout meet hoe goed de gereconstrueerde invoer lijkt op de originele invoer (MSE).
- KL-divergence dwingt de latente ruimte om dicht bij een normale verdeling te blijven.

---

De encoder neemt de invoer (spectrogram) en zet deze om naar een compactere representatie in de latente ruimte door middel van convolutionele lagen. Uiteindelijk berekent het `z_mean` en `z_log_var`, die de verdeling van de latente ruimte beschrijven.

In [7]:
# Encoder definition
input_shape = (513, 128, 1)  # Shape of the input (spectrogram)
latent_dim = 8  # Dimension of the latent space

# Input layer for spectrograms
inputs = Input(shape=input_shape)

# Convolutional layers to extract features
x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Flatten()(x)

# Fully connected layers that compute z_mean and z_log_var
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

# Sampling from the latent space
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Build the encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name="encoder")

- Conv2D-lagen extraheren patronen uit het spectrogram, zoals frequentie-inhoud en tijdsevolutie.
- Flatten maakt de data geschikt voor de volledig verbonden lagen.
- Dense-lagen berekenen de `z_mean` en `z_log_var`, die de latente ruimte beschrijven.

--- 

De decoder neemt de steekproef (`z`) uit de latente ruimte en reconstrueert het originele spectrogram door middel van transposed convolutionele lagen, die werken als omgekeerde convoluties.

In [8]:
# Decoder definition
decoder_input = Input(shape=(latent_dim,))

# Reshape the latent space back into the original spectrogram shape
x = Dense(513 * 128)(decoder_input)
x = Reshape((513, 128, 1))(x)

# Transposed convolutional layers to reconstruct the original spectrogram
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
outputs = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

# Build the decoder model
decoder = Model(decoder_input, outputs, name="decoder")

- **Dense** zet de latente ruimte om naar de grootte van het oorspronkelijke spectrogram.
- **Conv2DTranspose** werkt als een "omgekeerde" convolutie en bouwt het spectrogram laag voor laag opnieuw op.
- **Sigmoid** in de output-laag normaliseert de waarden tussen 0 en 1, wat past bij het genormaliseerde inputformaat van het model.

---

Tot slot compileren we het model met de Adam-optimizer en trainen we het door de reconstructiefout en KL-divergence te minimaliseren.

In [9]:
# VAE model (combining encoder and decoder)
vae = VAE(encoder, decoder)

# Compile the model using the Adam optimizer
vae.compile(optimizer='adam')

- **Adam-optimizer** wordt gebruikt voor het trainen van het model, omdat het goed werkt bij niet-lineaire optimalisatieproblemen zoals deze.

---

In deze stap trainen we de Variational Autoencoder (**VAE**) op de normale audiogegevens. We geven zowel de invoer (`X_train`) als de gewenste output (`X_train`) als hetzelfde, omdat het doel van een autoencoder is om de invoer te reconstrueren.

In [10]:
# Train the autoencoder on normal audio
vae.fit(X_train, X_train, epochs=20, batch_size=32)

Epoch 1/20
5/5 [==============================] - 6s 263ms/step - loss: 48116.6328
Epoch 2/20
5/5 [==============================] - 0s 86ms/step - loss: 56389.2266
Epoch 3/20
5/5 [==============================] - 0s 87ms/step - loss: 32691.9062
Epoch 4/20
5/5 [==============================] - 0s 87ms/step - loss: 2.7579
Epoch 5/20
5/5 [==============================] - 0s 88ms/step - loss: 0.0375
Epoch 6/20
5/5 [==============================] - 0s 87ms/step - loss: 0.0221
Epoch 7/20
5/5 [==============================] - 0s 87ms/step - loss: 0.0152
Epoch 8/20
5/5 [==============================] - 0s 87ms/step - loss: 0.0180
Epoch 9/20
5/5 [==============================] - 0s 87ms/step - loss: 0.0164
Epoch 10/20
5/5 [==============================] - 0s 87ms/step - loss: 0.0148
Epoch 11/20
5/5 [==============================] - 0s 86ms/step - loss: 0.0149
Epoch 12/20
5/5 [==============================] - 0s 87ms/step - loss: 0.0150
Epoch 13/20
5/5 [==============================]

### Samengevat
- De **Encoder** is verantwoordelijk voor het **comprimeren** van de oorspronkelijke spectogram naar de **latente  ruimte**. Deze latente ruimte is een lagere-dimensionale representatie (in dit geval bestaande uit 8 getallen) die de meest relevante informatie uit het spectrogram samenvat. De encoder zorgt ervoor dat het model leert wat de "essentie" is van het geluid zonder alle details vast te hoeven leggen. 

- De **Decoder** is verantwoordelijk voor het reconstrueren van het spectrogram vanuit de latente ruimte. Dit betekent dat het model probeert de oorspronkelijke input (het spectrogram) terug te krijgen, gebaseerd op die 8 getallen die uit de latente ruimte komen. De decoder gebruikt deze getallen als "startpunt" en bouwt laag voor laag het volledige spectrogram opnieuw op.

- De **loss-functie** bestaat uit twee delen: **reconstructiefout (MSE)** en **KL-divergence**. Samen zorgen deze voor de optimale balans tussen het nauwkeurig reconstrueren van de inputdata en het organiseren van de latente ruimte.
    - **Reconstructiefout (MSE)** meet hoe goed het model de oorspronkelijke input (het spectrogram) kan reconstrueren na compressie in de latente ruimte. Het vergelijkt het gereconstrueerde spectrogram met het originele spectrogram en berekent het kwadratische verschil tussen de twee. Een lagere reconstructiefout betekent dat het model goed in staat is om de belangrijkste informatie van het geluid vast te leggen en nauwkeurig te reconstrueren.
    - **KL-divergence** meet hoe goed de verdeling van de latente ruimte overeenkomt met een standaard normale verdeling. Het dwingt het model om de latente ruimte te structureren, zodat het een continuüm van mogelijke representaties vormt. Dit helpt het model bij het genereren van nieuwe data en zorgt ervoor dat de latente ruimte geordend blijft, wat belangrijk is voor de generalisatie van het model.
    - Samen zorgen **MSE** en **KL-divergence** ervoor dat het model zowel een nauwkeurige reconstructie kan maken als een gestructureerde latente ruimte behoudt. Het doel tijdens training is om de som van deze twee termen, de totale loss, te minimaliseren.

## Evaluation

Het grootste probleem met deze aanpak is de evaluation van het model. Er zijn geen labels om te concluderen wat goed en fout is. Daarom is er voor gekozen om te kijken naar verschillende thresholds om vervolgens hier live meet te testen.

In [11]:
# Reconstruct the training data (normal sounds) using the trained VAE
X_train_reconstructed = vae.predict(X_train)  # X_train contains the normal sounds used for training

# Calculate the Mean Squared Error (MSE) for the training data (normal sounds)
# MSE is calculated per sample by taking the mean squared difference between original and reconstructed data
mse_train = np.mean(np.power(X_train - X_train_reconstructed, 2), axis=(1, 2, 3))  # MSE for the normal data

# Set a threshold for anomaly detection based on the distribution of training reconstruction errors
# Here, the threshold is set to the mean of the MSE plus 2 standard deviations
threshold = np.mean(mse_train) + 2 * np.std(mse_train)  # Experiment with different threshold values if needed

# Output the chosen threshold value for anomaly detection
print(f"Threshold for anomaly detection: {threshold}")

# Output the distribution of reconstruction errors for the training data
print(f"Mean reconstruction error on training data: {np.mean(mse_train)}")
print(f"Minimal reconstruction error on training data: {np.min(mse_train)}")
print(f"Maximum reconstruction error on training data: {np.max(mse_train)}")

5/5 [==============================] - 0s 30ms/step
Threshold for anomaly detection: 0.020535769872367382
Mean reconstruction error on training data: 0.010228280909359455
Minimal reconstruction error on training data: 0.006618628278374672
Maximum reconstruction error on training data: 0.04694449156522751


In [12]:
# Set the parameters as used during live predection
sample_rate = 44100  # Sampling rate used to capture audio data, typically in samples per second
segment_duration = 2  # Duration of each audio segment in seconds (each audio fragment is 2 seconds long

In [13]:
# Declare global segment_samples before any processing
segment_samples = []

def audio_callback(indata, frames, time_info, status, audio_processor):
    global segment_samples  # Reference the global segment_samples

    if status:
        print(status)
    segment_samples.extend(indata[:, 0])  # Extend the list with new audio data

    if len(segment_samples) >= sample_rate * segment_duration:  # Check if enough audio has been collected
        current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        print(f"Prediction uitgevoerd op: {current_time}")

        # Process the audio into spectrogram features using the AudioProcessor
        audio_features = audio_processor.process_audio_live(segment_samples)

        # Check the shape of the live input (should be (1, 513, 128, 1))
        print(f"Shape of live input: {audio_features.shape}")

        # Predict anomalies using the VAE model
        reconstructed_audio = vae.predict(audio_features)

        # Calculate the reconstruction error (Mean Squared Error)
        mse = np.mean(np.power(audio_features - reconstructed_audio, 2))

        # Check for anomaly
        if mse > threshold:
            print(f"Anomalie gedetecteerd! Reconstructiefout: {mse}")
        else:
            print(f"Normaal geluid. Reconstructiefout: {mse}")

        # Reset the samples for the next batch
        segment_samples = []  # Reset the global segment_samples for the next batch

In [14]:
if __name__ == "__main__":
    # Initialize the AudioProcessor object
    audio_processor = AudioProcessor()

    print("Start streaming van audio...")
    with sd.InputStream(callback=lambda indata, frames, time_info, status: audio_callback(indata, frames, time_info, status, audio_processor),
                        channels=1, samplerate=sample_rate):
        try:
            while True:
                sd.sleep(100)  # Keep the process active while audio is streamed
        except KeyboardInterrupt:
            print("Audio streaming gestopt.")


Start streaming van audio...
Prediction uitgevoerd op: 2025-02-03 10:57:16
Shape of live input: (1, 513, 128, 1)
1/1 [==============================] - 0s 231ms/step
Normaal geluid. Reconstructiefout: 0.007363243028521538
input overflow
Prediction uitgevoerd op: 2025-02-03 10:57:18
Shape of live input: (1, 513, 128, 1)
1/1 [==============================] - 0s 17ms/step
Normaal geluid. Reconstructiefout: 0.007461633533239365
Prediction uitgevoerd op: 2025-02-03 10:57:20
Shape of live input: (1, 513, 128, 1)
1/1 [==============================] - 0s 18ms/step
Normaal geluid. Reconstructiefout: 0.007335428148508072
Prediction uitgevoerd op: 2025-02-03 10:57:22
Shape of live input: (1, 513, 128, 1)
1/1 [==============================] - 0s 18ms/step
Normaal geluid. Reconstructiefout: 0.018336864188313484
Prediction uitgevoerd op: 2025-02-03 10:57:24
Shape of live input: (1, 513, 128, 1)
1/1 [==============================] - 0s 18ms/step
Normaal geluid. Reconstructiefout: 0.01304662413895

In [ ]:
import tensorflow as tf

# Direct converteren naar TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(vae)  # Gebruik je VAE-model direct
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

tflite_model = converter.convert()

# Opslaan als .tflite-bestand
with open("models/vae_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model succesvol opgeslagen als vae_model.tflite")


INFO:tensorflow:Assets written to: C:\Users\Paul\AppData\Local\Temp\tmpxne45mx3\assets


INFO:tensorflow:Assets written to: C:\Users\Paul\AppData\Local\Temp\tmpxne45mx3\assets


Model succesvol opgeslagen als vae_model.tflite


Dit zegt natuurlijk niks over hoe goed het model werkt, want het is live testing. Maar deze aanpak is redelijk succesvol. Wanneer ik in mijn handen klap of tegen de microfoon tik dan ziet hij deze als een abnormaliteit.

---

## Deployment
**Conclusie:**
- De huidige aanpak lijkt hoopvol en deze moeten we testen op de Raspberry Pi 

**Volgende Stappen:**
- Data Augmentatie toepassen
- Beter idee krijgen hoe de data eruit moet zien voor dit model
- Kijken welke aanpassesn een positief gevolg heeft voor het resultaat